In [1]:
import pandas
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
NER_path = 'multinerd_en.tsv'
NER_test = 'NER-final-test.tsv'

In [2]:
#NER

NER_dataset = pandas.read_csv(NER_path,sep='\t',error_bad_lines=False,encoding='utf8',header=None,quoting=3)

df_test = pandas.read_csv(NER_test,sep='\t',error_bad_lines=False,encoding='utf8',quoting=3)

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df = NER_dataset
df = df[[0,1,2]]
df.columns= ['token id','token','BIO NER tag']
df_train = df[:100000]
df[850:860]

,token id,token,BIO NER tag
850,11,"""",O
851,12,",",O
852,13,"""",O
853,14,Die,B-MEDIA
854,15,Fledermaus,I-MEDIA
855,16,"""",O
856,17,",",O
857,18,and,O
858,19,Ravel,B-PER
859,20,’s,O


In [4]:
training_features=[]
training_gold_labels=[]

for ind in df_train.index:
    a_dict = {
        'words':df_train['token'][ind]
    }
    training_features.append(a_dict)
    training_gold_labels.append(df_train['BIO NER tag'][ind])
    
test_features=[]
test_gold_labels=[]

for ind in df_test.index:
    a_dict = {
        'words':df_test['token'][ind]
    }
    test_features.append(a_dict)
    test_gold_labels.append(df_test['BIO NER tag'][ind])

In [5]:
vec = DictVectorizer()
an_array = training_features + test_features

the_array = vec.fit_transform(an_array)

new_training = the_array[:100000]
new_test = the_array[100000:]

In [6]:
lin_clf = svm.LinearSVC()

In [7]:
from collections import Counter 
Counter(training_gold_labels),Counter(test_gold_labels)

(Counter({'O': 87144,
          'B-PER': 2525,
          'I-PER': 2626,
          'B-LOC': 1565,
          'B-MEDIA': 308,
          'I-MEDIA': 519,
          'B-TIME': 84,
          'B-EVE': 109,
          'I-EVE': 174,
          'I-TIME': 68,
          'I-LOC': 762,
          'B-ORG': 838,
          'I-ORG': 1146,
          'B-PLANT': 213,
          'B-ANIM': 339,
          'I-ANIM': 212,
          'I-PLANT': 78,
          'B-FOOD': 296,
          'B-DIS': 427,
          'I-DIS': 316,
          'B-CEL': 51,
          'B-SUPER': 12,
          'I-SUPER': 3,
          'I-FOOD': 112,
          'B-VEHI': 6,
          'I-VEHI': 4,
          'B-INST': 8,
          'I-INST': 8,
          'I-CEL': 33,
          'B-PHY': 2,
          'I-PHY': 3,
          'B-BIO': 7,
          'I-BIO': 2}),
 Counter({'O': 183,
          'B-ORG': 4,
          'I-ORG': 3,
          'B-LOC': 4,
          'B-MISC': 3,
          'B-PER': 6,
          'I-PER': 8,
          'I-MISC': 1,
          'I-LOC': 2}))

In [10]:
lin_clf.fit(new_training, training_gold_labels) 
predicted_labels = lin_clf.predict(new_test)

NERC_report = classification_report(test_gold_labels, predicted_labels,output_dict=True)

NERC_cr_df = pandas.DataFrame(NERC_report).transpose()
NERC_cr_df

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_cla

,precision,recall,f1-score,support
B-ANIM,0.000000,0.000000,0.000000,0.000000
B-LOC,0.400000,0.500000,0.444444,4.000000
B-MISC,0.000000,0.000000,0.000000,3.000000
B-ORG,0.333333,0.250000,0.285714,4.000000
B-PER,1.000000,0.500000,0.666667,6.000000
I-LOC,0.500000,0.500000,0.500000,2.000000
I-MEDIA,0.000000,0.000000,0.000000,0.000000
I-MISC,0.000000,0.000000,0.000000,1.000000
I-ORG,0.500000,0.666667,0.571429,3.000000
I-PER,1.000000,0.250000,0.400000,8.000000


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
#Sentiment
# senti_path_finance = 'senti-data.csv'
# senti_path_movie = 'IMDB_Dataset.csv'
senti_test = 'sentiment-topic-final-test.tsv'
# senti_twitter ='Twitter_Data.csv'
senti_reddit ='Reddit_Data.csv'

In [13]:
# df_senti_movie = pandas.read_csv(senti_path_movie,encoding='utf8')
# df_senti_finance = pandas.read_csv(senti_path_finance,encoding='latin-1',header=None)
df_senti_test = pandas.read_csv(senti_test,sep='\t',error_bad_lines=False,encoding='utf8',quoting=3)

df_senti_reddit = pandas.read_csv(senti_reddit,encoding='utf8')
# df_senti_twitter = pandas.read_csv(senti_twitter,encoding='utf8')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
df_senti_reddit.rename(columns={'clean_comment':'Sentence','category':'Sentiment'},inplace = True)
df_senti_reddit['Sentiment']=df_senti_reddit['Sentiment'].replace({-1:'negative',0:'neutral',1:'positive'})
df_senti_reddit = df_senti_reddit.dropna(axis=0)

In [15]:
df_senti_reddit['Sentiment'].value_counts()

positive    15830
neutral     13042
negative     8277
Name: Sentiment, dtype: int64

In [16]:
df_senti_test.rename(columns={'text':'Sentence','sentiment':'Sentiment'},inplace=True)
# df_senti_test
df_senti_test = df_senti_test[['Sentence','Sentiment']]
df_senti_test

,Sentence,Sentiment
0,It took eight years for Warner Brothers to rec...,negative
1,All the New York University students love this...,positive
2,This Italian place is really trendy but they h...,negative
3,"In conclusion, my review of this book would be...",positive
4,The story of this movie is focused on Carl Bra...,neutral
5,Chris O'Donnell stated that while filming for ...,neutral
6,My husband and I moved to Amsterdam 6 years ag...,positive
7,Dame Maggie Smith performed her role excellent...,positive
8,The new movie by Mr. Kruno was shot in New Yor...,neutral
9,"I always have loved English novels, but I just...",negative


In [17]:
# df_senti_movie = df_senti_movie.sample(5000)
# df_senti_movie = df_senti_movie['Sentence'].str.replace("^\\<br>$","")
df_senti = pandas.concat([df_senti_reddit, df_senti_test])
# df_senti = pandas.concat([df_senti_movie, df_senti_test])
df_senti['Sentiment'].value_counts()

positive    15834
neutral     13045
negative     8280
Name: Sentiment, dtype: int64

In [26]:
vectorizer = TfidfVectorizer(min_df=2, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english'))

# vectorizer = TfidfVectorizer (min_df=5, 
#                               tokenizer = nltk.word_tokenize,
#                               stop_words=stopwords.words('english'))
combined_data = df_senti

In [27]:
training_data_senti = df_senti[['Sentence','Sentiment']]
test_data_senti = df_senti_test[['Sentence','Sentiment']]
# print(len(combined_data))
# break
new_combined_sentence = vectorizer.fit_transform(combined_data.Sentence).toarray()
print(new_combined_sentence.shape)
new_training_senti = new_combined_sentence[:37149]
new_test_senti = new_combined_sentence[37149:]
y_new_training = combined_data.Sentiment[:37149]
y_new_test = combined_data.Sentiment[37149:]



/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(37159, 25602)


In [28]:
print(new_test_senti.shape)

(10, 25602)


In [29]:
clf = MultinomialNB().fit(new_training_senti,y_new_training)

In [30]:
y_pred = clf.predict(new_test_senti)

In [31]:
SENTI_report = classification_report(y_new_test,y_pred,output_dict=True)

SENTI_cr_df = pandas.DataFrame(SENTI_report).transpose()
SENTI_cr_df

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
negative,0.000000,0.000000,0.00,3.0
neutral,0.500000,0.333333,0.40,3.0
positive,0.375000,0.750000,0.50,4.0
accuracy,0.400000,0.400000,0.40,0.4
macro avg,0.291667,0.361111,0.30,10.0
weighted avg,0.300000,0.400000,0.32,10.0


In [32]:
def important_features_per_class(vectorizer,classifier,n=10): #n is the number of top features
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:
important_features_per_class(vectorizer, clf)

Important words in negative documents
negative 154.21486876002615 fuck
negative 143.67658214449716 bjp
negative 133.73004863759246 people
negative 130.7966415692015 modi
negative 124.86366680580251 shit
negative 123.90423875675283 like
negative 118.41284544317223 india
negative 104.93082250961409 fucking
negative 91.6900416140093 one
negative 86.29371424730641 bad
-----------------------------------------
Important words in neutral documents
neutral 176.93981752532682 modi
neutral 130.1582944469837 bjp
neutral 119.03731780256194 india
neutral 112.45565503090161 like
neutral 111.85639526339759 hai
neutral 78.71411704028769 one
neutral 72.72255398375265 get
neutral 66.96004062040977 people
neutral 63.9677560704131 congress
neutral 61.790049718723964 time
-----------------------------------------
Important words in positive documents
positive 325.53183184110674 good
positive 261.55865904353 people
positive 260.85464240083763 india
positive 250.45549344130848 bjp
positive 248.6796574449823